In [1]:
from util import get_freq
import matplotlib.pyplot as plt
import numpy as np
from random import uniform
from NPSmethods import readInImages
from NPSmethods import pupilFunc
from numpy.fft import ifftshift, ifft2
import pickle as pkl

In [2]:
#Geranate random assortment of parameters
def make_Paras(seedParas, numToGen):
    outPara = [None] * len(seedParas)
    paraTemp = [None] * len(seedParas)
    for i in range(numToGen):
        for j in range(len(seedParas)):
            para = seedParas[j]
            if j ==3 or j == 5 or j == 6:
                para = uniform(para*.5, para*2)
            paraTemp[j] = para
        outPara = np.vstack((outPara, paraTemp))
    return outPara
        

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = get_freq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    
    return M2k_Fit

#Generate fake OD images with random noise
def generate_Noise(M2k_Fit, idealOD, numIms):
    M2k_Fit *= idealOD.sum()
    shape = np.shape(M2k_Fit)
    NPSs_avg = np.empty_like(M2k_Fit)
    M2k_Fit = M2k_Fit.flatten()

    for i in range(numIms):
        NPSs_vals = np.random.dirichlet(np.ones(len(M2k_Fit[M2k_Fit != 0])), size = 1)
        NPSs_temp = np.zeros_like(M2k_Fit)

        for j in range(len(np.nonzero(M2k_Fit)[0])):
            NPSs_temp[np.nonzero(M2k_Fit)[0][j]] = NPSs_vals[0][j] * np.sum(M2k_Fit)

        NPSs_temp = np.reshape(NPSs_temp, shape)
        NPSs_avg = np.dstack((NPSs_avg, NPSs_temp))

    NPSs_avg = NPSs_avg.T
    NPSs_avg = np.delete(NPSs_avg, 0, 0)
    fakeODs = np.empty_like(NPSs_avg[0])
    
    for NPS in NPSs_avg:
        NPS = np.sqrt(np.abs(NPS)) * np.random.choice((-1, 1))
        noise = ifftshift(ifft2(ifftshift(NPS)))
        noise = np.abs(noise)
        fakeODs = np.dstack((fakeODs, np.exp(-(noise + idealOD))))
        
    fakeODs = fakeODs.T
    fakeODs = np.delete(fakeODs, 0, 0)
    return fakeODs

In [ ]:
folderpath = './raw_image/141902/'
savepath = './results/141902_101-123/'
seedParas = [0.35060326843236184, 0.6484441665068852, -2.63564159087613, 0.5094606166480791, -1.8773441762223027, 0.8206242586655179, 1.0257364487180909]

numIms = 50
numParas = 400

fakeODs = np.empty(shape = (numIms, 100, 100))
_, idealOD, _, _, _, _ = readInImages(folderpath, 23, 0, trapRegion = (slice(100, 200), slice(100, 200)), noiseRegion = (slice(0, 300), slice(0, 300)))

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
} 

i = 0
for paras in make_Paras(seedParas, 400)[1:-1]:
    i += 1
    print(i)
    fakeODs = np.append(fakeODs, generate_Noise(make_M2k_Fit(paras, imgSysData), idealOD, numIms), axis = 0)
fakeODs = fakeODs[numIms:]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204


In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(fakeODs[0])

In [ ]:
pkl.dump(fakeODs, open(savepath + "fakeODs", "wb"))